In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from datasets import load_dataset
from diffusers import DDIMScheduler, DDPMScheduler, DDPMPipeline
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm

c:\Users\mharu\miniconda3\envs\aud2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = (
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

In [3]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is DC31-00BD

 Directory of c:\Users\mharu\projects\aud2image

16.05.2025  20:00    <DIR>          .
12.05.2025  20:06    <DIR>          ..
16.05.2025  19:25             3.760 .gitignore
16.05.2025  19:21    <DIR>          dataset
16.05.2025  19:33    <DIR>          ddpm-prof-sauer-spectogram-256
10.05.2025  14:13    <DIR>          ddpm-spectogram-256
16.05.2025  19:58         1.037.086 diffusion.ipynb
16.05.2025  18:15    <DIR>          final
10.05.2025  14:24    <DIR>          generated
10.05.2025  15:32    <DIR>          generated2
16.05.2025  20:00        28.228.221 inference.ipynb
12.05.2025  19:48    <DIR>          maestro
16.05.2025  19:25            92.724 Mel.ipynb
15.05.2025  17:47    <DIR>          music
16.05.2025  20:30            14.409 pre_trained.ipynb
16.05.2025  19:25        35.746.863 preprocessing.ipynb
16.05.2025  17:34    <DIR>          prof_sauer
16.05.2025  19:25         1.942.229 spec2music.ipynb
16.05.2025

In [4]:
image_pipe = DDPMPipeline.from_pretrained("ddpm-spectogram-256")
image_pipe.to(device);

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch ddpm-spectogram-256\unet: Error no file named diffusion_pytorch_model.safetensors found in directory ddpm-spectogram-256\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  9.07it/s]


In [ ]:
scheduler = DDPMScheduler.from_config("ddpm-spectogram-256/scheduler/scheduler_config.json")

In [ ]:
image_size = 256  # @param
batch_size = 4  # @param

In [ ]:
from datasets import load_dataset, Dataset
from datasets import Image
import os


# Path to your local image folder
image_folder = "music/images"

# Load images into Hugging Face dataset format
dataset = load_dataset("imagefolder", data_dir=image_folder, split="train")

# Cast to Image feature
dataset = dataset.cast_column("image", Image())

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        #transforms.Resize((config.image_size, config.image_size)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5]),
    ]
)

In [ ]:
def transform(examples):
    images = [preprocess(image) for image in examples["image"]]
    return {"images": images}


dataset.set_transform(transform)

In [ ]:
import torch

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print("Previewing batch:")
batch = next(iter(train_dataloader))
grid = torchvision.utils.make_grid(batch["images"], nrow=4)
plt.imshow(grid.permute(1, 2, 0).cpu().clip(-1, 1) * 0.5 + 0.5);

In [20]:
num_epochs = 2  # @param
lr = 1e-5  # 2param
grad_accumulation_steps = 2  # @param

optimizer = torch.optim.AdamW(image_pipe.unet.parameters(), lr=lr)

losses = []

for epoch in range(num_epochs):
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        clean_images = batch["images"].to(device)
        # Sample noise to add to the images
        noise = torch.randn(clean_images.shape).to(clean_images.device)
        bs = clean_images.shape[0]

        # Sample a random timestep for each image
        timesteps = torch.randint(
            0,
            image_pipe.scheduler.num_train_timesteps,
            (bs,),
            device=clean_images.device,
        ).long()

        # Add noise to the clean images according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        noisy_images = image_pipe.scheduler.add_noise(clean_images, noise, timesteps)

        # Get the model prediction for the noise
        noise_pred = image_pipe.unet(noisy_images, timesteps, return_dict=False)[0]

        # Compare the prediction with the actual noise:
        loss = F.mse_loss(
            noise_pred, noise
        )  # NB - trying to predict noise (eps) not (noisy_ims-clean_ims) or just (clean_ims)

        # Store for later plotting
        losses.append(loss.item())

        # Update the model parameters with the optimizer based on this loss
        loss.backward(loss)

        # Gradient accumulation:
        if (step + 1) % grad_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    print(
        f"Epoch {epoch} average loss: {sum(losses[-len(train_dataloader):])/len(train_dataloader)}"
    )

# Plot the loss curve:
plt.plot(losses)

 30%|██▉       | 35/118 [09:19<22:07, 15.99s/it]


KeyboardInterrupt: 